In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
from math import *
import time

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [4]:
# defination of activation function
def activation(x):
    return (torch.sin(x))**3 

In [5]:
# Build ResNet with three blocks
class net(torch.nn.Module):
    def __init__(self,input_width,layer_width):
        super(net,self).__init__()
        self.layer_in = torch.nn.Linear(input_width, layer_width)
        self.layer1 = torch.nn.Linear(layer_width, layer_width)
        self.layer2 = torch.nn.Linear(layer_width, layer_width)
        self.layer3 = torch.nn.Linear(layer_width, layer_width)
        self.layer4 = torch.nn.Linear(layer_width, layer_width)
        self.layer5 = torch.nn.Linear(layer_width, layer_width)
        self.layer6 = torch.nn.Linear(layer_width, layer_width)
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x):
        y = self.layer_in(x)
        y = y + activation(self.layer2(activation(self.layer1(y)))) # residual block 1
        y = y + activation(self.layer4(activation(self.layer3(y)))) # residual block 2
        y = y + activation(self.layer6(activation(self.layer5(y)))) # residual block 3
        output = self.layer_out(y)
        return output

In [6]:
dimension = 4

In [7]:
# Defination of exact solution
def u_ex(x):  
    x_norm = torch.norm(x, dim = 1) # compute norm of x; dim = 1 - > sum by row
    u_x = torch.sin(pi/2*(1-x_norm)).reshape([x.size()[0],1])
    return u_x

In [8]:
# Defination of f(x)
def f(x):
    x_norm = torch.norm(x, dim = 1)
    u_temp = u_ex(x)
    f_temp = 1/4*pi**2*torch.sin(pi/2*(1-x_norm)) + 1/2*pi*(dimension-1)/x_norm*torch.cos(pi/2*(1-x_norm))  
    return f_temp.reshape([x.size()[0],1])+ u_temp**3

In [9]:
# Generate points by random
data_size = 1000
def generate_data():
    i = 0
    x = torch.tensor([])
    while x.size()[0] < data_size:
        temp_x = 2*torch.rand(data_size, dimension) - 1
        index = torch.where(torch.norm(temp_x, dim = 1) < 1)
        temp_x = temp_x[index]
        x = torch.cat((x,temp_x),0)
    x = x[1:data_size, :]
    return x.cuda()

In [10]:
# Defination of loss function for DRM
def loss_function(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) 
    f_temp = f(x)
    part_1 = torch.sum(1/2*uxsq + 1/4*u_hat**4 - f_temp*u_hat)/x.size()[0]
    
    Nb = 200
    y = (2*torch.rand(Nb, dimension)-1).cuda()
    y_norm = torch.norm(y, dim = 1).cuda()
    for i in range(dimension):
        y[:, i] = y[:, i] / y_norm
    part_2 = (torch.sum((model(y)-u_ex(y))**2)/y.size()[0])
    
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 

In [11]:
# Parameters initialization by normal distribution for neural networks
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m,nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = net(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1" )
model.to(device)

net(
  (layer_in): Linear(in_features=4, out_features=8, bias=True)
  (layer1): Linear(in_features=8, out_features=8, bias=True)
  (layer2): Linear(in_features=8, out_features=8, bias=True)
  (layer3): Linear(in_features=8, out_features=8, bias=True)
  (layer4): Linear(in_features=8, out_features=8, bias=True)
  (layer5): Linear(in_features=8, out_features=8, bias=True)
  (layer6): Linear(in_features=8, out_features=8, bias=True)
  (layer_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
# Defination of relative error
def relative_error():
    x = generate_data()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [15]:
traintime = 10000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [16]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = generate_data()
    x.requires_grad = True
    loss = loss_function(x)
    loss.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Dirichlet_nonlinear_sin3.npy", error_save)
np.save("DRM_relative_error_4D_Dirichlet_nonlinear_sin3.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(41.1449, device='cuda:1')
current relative error is:  tensor(2.0116, device='cuda:1')
current epoch is:  50
current loss is:  tensor(5.4228, device='cuda:1')
current relative error is:  tensor(1.0720, device='cuda:1')
current epoch is:  100
current loss is:  tensor(2.6407, device='cuda:1')
current relative error is:  tensor(0.9776, device='cuda:1')
current epoch is:  150
current loss is:  tensor(1.7650, device='cuda:1')
current relative error is:  tensor(0.9254, device='cuda:1')
current epoch is:  200
current loss is:  tensor(1.0069, device='cuda:1')
current relative error is:  tensor(0.8909, device='cuda:1')
current epoch is:  250
current loss is:  tensor(1.0490, device='cuda:1')
current relative error is:  tensor(0.8781, device='cuda:1')
current epoch is:  300
current loss is:  tensor(0.5526, device='cuda:1')
current relative error is:  tensor(0.8493, device='cuda:1')
current epoch is:  350
current loss is:  tensor(0.4967, device='cuda:1'

current epoch is:  3050
current loss is:  tensor(-1.2283, device='cuda:1')
current relative error is:  tensor(0.1110, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-1.2203, device='cuda:1')
current relative error is:  tensor(0.0976, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-1.0606, device='cuda:1')
current relative error is:  tensor(0.1116, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-1.1110, device='cuda:1')
current relative error is:  tensor(0.0992, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-1.1883, device='cuda:1')
current relative error is:  tensor(0.1056, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-1.3168, device='cuda:1')
current relative error is:  tensor(0.1022, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-1.1215, device='cuda:1')
current relative error is:  tensor(0.1016, device='cuda:1')
current epoch is:  3400
current loss is:  tensor(-1.171

current epoch is:  6100
current loss is:  tensor(-1.1287, device='cuda:1')
current relative error is:  tensor(0.1191, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-1.0867, device='cuda:1')
current relative error is:  tensor(0.1306, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-1.1083, device='cuda:1')
current relative error is:  tensor(0.0741, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-1.1044, device='cuda:1')
current relative error is:  tensor(0.1105, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-1.2686, device='cuda:1')
current relative error is:  tensor(0.1119, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-1.1824, device='cuda:1')
current relative error is:  tensor(0.1130, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-1.3244, device='cuda:1')
current relative error is:  tensor(0.1137, device='cuda:1')
current epoch is:  6450
current loss is:  tensor(-1.198

current epoch is:  9150
current loss is:  tensor(-1.2104, device='cuda:1')
current relative error is:  tensor(0.0926, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-1.3799, device='cuda:1')
current relative error is:  tensor(0.1133, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-1.0807, device='cuda:1')
current relative error is:  tensor(0.1005, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-1.3062, device='cuda:1')
current relative error is:  tensor(0.1241, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-1.3440, device='cuda:1')
current relative error is:  tensor(0.1029, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-1.1455, device='cuda:1')
current relative error is:  tensor(0.1152, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-1.2915, device='cuda:1')
current relative error is:  tensor(0.1222, device='cuda:1')
current epoch is:  9500
current loss is:  tensor(-1.286